# Tracking The Space Station In Real Time

<span class="pubdate">April 2016</span>

Do you know where the [International Space Station](http://www.nasa.gov/audience/forstudents/5-8/features/nasa-knows/what-is-the-iss-58.html) is right now? There are lots of sites[^1] maps of the current position of the ISS. NASA probably knows where it is[^2], but how is it done?

[^1]: <https://www.google.com/webhp?q=The+current+position+of+the+ISS>
[^2]: Citation Needed

## Early History Of Satellite Tracking

In the 1950's scientists and engineers (and science fiction writters) realized that launching a rocket into orbit would be practical in the near future. There were still many problems to solve before the first spacecraft would fly, and tracking was high on the list.

Once a rocket was launched, you could send back data about how the mission was going via radios and recievers on the ground. But once it gets out over the ocean and below the horizon there's no way to hear from it! The effect is that you launch a rocket, everything is going well, near the end of the burn the vehicle goes below the horizon and dissapears off radar and ... who knows! You wait an hour and a half for it to finish it's first orbit and hope to hear from it again. Or not. It's a pure information blackout. So how will you find it again when it comes around. You can't be 100% sure that it ended up in the entended orbit.

It was known that in the right lighting conditions (in the early evening or morning when there would be sunlight on the craft after it starts to get dark on the ground) you could see and track an object optically, getting very high precision timing and angle data for a few minutes --- enough to compute an orbit trajectory. The early tracking stations used to practice this before the first satellite was even launched by dangling a lightbulb on a cord behind an airplane! But they found it was nearly impossible to find that dot of light unless you already had a pretty good idea where and when it was going to appear.


### Radios To The Rescue

The answer was to put a rudamentry transponder on the craft and use a sensitive wide-angle radio dish on the ground that could pick out the signal as long as it went overhead. The rough orbit could be determined via this radio method and high quality data could be picked up later with the optical method. This first system was called Minitrack[^3] and was used by the United States for tracking all the early satellites.


### Radar

As radar and radio processing hardware got more and more advanced it started to become possible to direcly 'see' satellites overhead with strong radar systems. This meant you could redily track otherwise silent objects (like satellites from rival millitaries, or space debris). Today the US operates large radar tracking systems and so-called "radio fences" the throw up thousands of watts of long-wave radio and listen for reflections off of anything in the sky. Optical techniques are still used as well, with both ground based and space based telescopes (a hold over from the Cold War) being used to regularly track objects. The United States Strategic Command currently identified, and tracks upwards of 20,000 objects larger than 10 centimeters.

### DIY

For satellites that are still in good working order, it's possible to determine the location onboard and simply have it tell us where it is. For the International Space Station this is exactly what is done. A pair of high quality GPS receivers (plus more for redundancy) are operated continuously and used for the primary location and attitude of the station. The location, velocity and therefore expected orbit is continuously monitored and beamed down in telemetry to mission control. If that fails, we have the most recent orbit data that would be accurate for many hours, up to a few days. In the mean time radar and optical tracking can fill in the gaps. Though it should be noted that GPS is very reliable in Low Earth Orbit (despite the comon misconception that it wouldn't work in space).

[^3]: <https://en.wikipedia.org/wiki/Minitrack>

## Getting The Data From NASA

A few years ago NASA ran a fun project called ISS LIVE! that used some javascript streaming protocols to show near-real-time data from the servers in mission control. One of the pieces of data were state vectors. Gathering ISS state vector data from [isslive.com](http://isslive.com/displays/adcoDisplay1.html).

## The Data

In [ ]:
# dependencies
from numpy import loadtxt, column_stack, subtract, multiply, divide, average, degrees, delete
from numpy.linalg import norm
from numpy import mgrid, pi, sin, cos
from sgp4.earth_gravity import wgs72 as gravity_model
from sgp4.ext import rv2coe
import datetime
import locale
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

# constants
GPS2UNIX_OFFSET  = 315964800  # sec
RADIUS_EARTH     =      6371  # km

# Useful date formats for x-axis
date_fmt_long = mdates.DateFormatter('%b-%d %H:%M')
date_fmt_day = mdates.DateFormatter('%b %d')

#dates
mar1 = datetime.datetime(2016,3,1)
mar31 = datetime.datetime(2016,3,31)
march = [mar1, mar31]

# colors
red = "#941e56"
blue = "#4a73e1"
green = "#22b464"

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

u, v = mgrid[0:2*pi:40j, 0:pi:20j]
radius = RADIUS_EARTH
es_x = cos(u)*sin(v)*radius
es_y = sin(u)*sin(v)*radius
es_z = cos(v)*radius

In [ ]:
# import data
columns = loadtxt('./data/ISS-Live-Capture-March.csv', delimiter=',', unpack=True)

receive_time = columns[0]
state_time   = columns[1]
state_x, state_y, state_z = columns[2:5]
state_vx, state_vy, state_vz = columns[5:8]
number_of_points = len(state_time)

# converty GPS stamp to datetime object
state_time = [datetime.datetime.utcfromtimestamp(t + GPS2UNIX_OFFSET) for t in state_time]

# vector position
state_vec = column_stack((state_x,  state_y,  state_z))
radius = norm(state_vec, axis=1)

# vector velocity
state_velvec = column_stack((state_vx,  state_vy,  state_vz))
velocity = norm(state_velvec, axis=1)

# Orbital Elements
a = []
e = []
incl = []
for i, t in enumerate(state_time):
    p, sm_a, ecc, inc, omega, argp, nu, m, arglat, truelon, lonper = rv2coe(state_vec[i], state_velvec[i]/1000.0, gravity_model.mu)
    a.append(sm_a)
    e.append(ecc)
    incl.append(degrees(inc))

# Throw out weird looking state vectors
badlist = []
for i, t in enumerate(state_time):
    # weird timestamps
    if state_time[i] < datetime.datetime(2016, 2, 28, 0,0,0):
        badlist.append(i)
    if state_time[i] > datetime.datetime(2016, 4, 2, 0,0,0):
        badlist.append(i)
        
    # wild position
    if radius[i] > 6788.0 or radius[i] < 6774.5:
        badlist.append(i)
        
    # wild velocity 
    if velocity[i] > 7674.0 or velocity[i] < 7657.0:
        badlist.append(i)
        
    # orbital params don't look right
    if a[i] > 6791.0 or a[i] < 6776: badlist.append(i)
    if e[i] > 0.0016: badlist.append(i)
        
        
# ignore dupes
badlist = set(badlist)


state_time   = [value for i, value in enumerate(state_time)   if i not in badlist]
state_x      = [value for i, value in enumerate(state_x)      if i not in badlist]
state_y      = [value for i, value in enumerate(state_y)      if i not in badlist]
state_z      = [value for i, value in enumerate(state_z)      if i not in badlist]
state_vx     = [value for i, value in enumerate(state_vx)     if i not in badlist]
state_vy     = [value for i, value in enumerate(state_vy)     if i not in badlist]
state_vz     = [value for i, value in enumerate(state_vz)     if i not in badlist]
state_vec    = [value for i, value in enumerate(state_vec)    if i not in badlist]
radius       = [value for i, value in enumerate(radius)       if i not in badlist]
state_velvec = [value for i, value in enumerate(state_velvec) if i not in badlist]
velocity     = [value for i, value in enumerate(velocity)     if i not in badlist]
a            = [value for i, value in enumerate(a)            if i not in badlist]
e            = [value for i, value in enumerate(e)            if i not in badlist]
incl         = [value for i, value in enumerate(incl)         if i not in badlist]

# averages
avg_velocity = average(velocity)
avg_a = average(a)
avg_e = average(e)

# find the gaps in the time
state_gaps = []
for i, t in enumerate(state_time[:-1]):
    diff = (state_time[i+1] - t).seconds
    if diff > 15:
        state_gaps.append(i)

In [ ]:
print """The data coverage is from %s through %s (%d days). There are %s state vectors
in the file, though %s were determined to be odd, and not used in this analysis. In addition the reporting service
failed often during the data gathering, so there are significant portions of data missing.
""" % (state_time[0].strftime("%B %d, %H:%M:%S UTC"),
       state_time[-1].strftime("%B %d, %H:%M:%S UTC"),
       (state_time[-1] - state_time[0]).days,
       locale.format("%d", number_of_points, grouping=True),
       locale.format("%d", len(badlist), grouping=True))

Download the raw data:

 - This article, source code, and all data: [archive.zip](https://github.com/natronics/ISS-TLE-Propagation/archive/gh-pages.zip), [archive.tar.gz](https://github.com/natronics/ISS-TLE-Propagation/archive/gh-pages.tar.gz)
 - Visit the [GitHub project page](https://github.com/natronics/ISS-TLE-Propagation)

See the position vector over time

In [ ]:
fig, ax1 = plt.subplots(figsize=(24,8))
plt.title(r"ISS State Vector J2000 X,Y,Z Position Over Time")
plt.ylabel(r"J2000 Coordinate [km]")
plt.xlabel(r"State Vector Date [Month Day]")

begin_data = 0
for end_data in state_gaps:
    plt.plot(state_time[begin_data:end_data], state_x[begin_data:end_data], color=red, alpha=0.75)
    plt.plot(state_time[begin_data:end_data], state_y[begin_data:end_data], color=blue, alpha=0.75)
    plt.plot(state_time[begin_data:end_data], state_z[begin_data:end_data], color=green, alpha=0.75)
    begin_data = end_data+1

# hack for labels
plt.plot([state_time[0],state_time[0]], [-100000,-110000], color=red, alpha=0.8, label="X")
plt.plot([state_time[0],state_time[0]], [-100000,-110000], color=blue, alpha=0.8, label="Y")
plt.plot([state_time[0],state_time[0]], [-100000,-110000], color=green, alpha=0.8, label="Z")

ax1.xaxis.set_major_formatter(date_fmt_day)
plt.ylim([-10e3,10e3])
plt.xlim(march)
ax1.legend(loc=1, title="Coordinate:")
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(24,8))
plt.title(r"ISS State Vector J2000 X,Y,Z Position, One Orbit (typical)")
plt.ylabel(r"J2000 Coordinate [km]")
plt.xlabel(r"State Vector Date [Month-Day HH:MM (UTC)]")

begin_data = 0
for end_data in state_gaps:
    plt.plot(state_time[begin_data:end_data], state_x[begin_data:end_data], color=red, alpha=0.75)
    plt.plot(state_time[begin_data:end_data], state_y[begin_data:end_data], color=blue, alpha=0.75)
    plt.plot(state_time[begin_data:end_data], state_z[begin_data:end_data], color=green, alpha=0.75)
    begin_data = end_data+1

# hack for labels
plt.plot([state_time[0],state_time[0]], [-100000,-110000], color=red, alpha=0.8, label="X")
plt.plot([state_time[0],state_time[0]], [-100000,-110000], color=blue, alpha=0.8, label="Y")
plt.plot([state_time[0],state_time[0]], [-100000,-110000], color=green, alpha=0.8, label="Z")

start_date = datetime.datetime(2016,3,2,0,59,0)
end_date   = datetime.datetime(2016,3,2,2,32,0)
ax1.xaxis.set_major_formatter(date_fmt_long)
plt.setp(ax1.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.ylim([-10e3, 10e3])
plt.xlim([start_date, end_date])
ax1.legend(loc=1, title="Coordinate:")
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(24,24))
ax1 = plt.axes(projection='3d')
plt.title(r"ISS Orbit")
plt.xlabel(r"J2000 X [km]")
plt.ylabel(r"J2000 Y [km]")
ax1.set_zlabel('J2000 Z [km]')

ax1.plot_wireframe(es_x, es_y, es_z, color=blue, alpha=0.2, lw=1.2)

begin_data = 0
for end_data in state_gaps:
    ax1.plot(state_x[begin_data:end_data], state_y[begin_data:end_data], state_z[begin_data:end_data], '-', color=red, alpha=0.7, lw=0.4)
    begin_data = end_data+1

ax1.view_init(elev=15.0, azim=60)
plt.setp(ax1.get_zticklabels(), horizontalalignment='right')
ax1.set_xlim(-6000, 6000)
ax1.set_ylim(-6000, 6000)
ax1.set_zlim(-6000, 6000)
ax1.set_aspect('equal','box')
plt.show()

See the alititude over time

In [ ]:
height = subtract(radius, RADIUS_EARTH)

avg_height = average(height)
fig, ax1 = plt.subplots(figsize=(24,8))
plt.title(r"ISS Geocentric Altitude Over Time")
plt.ylabel(r"Geocentric Altitude [km]")
plt.xlabel(r"State Vector Date [Month Day]")

begin_data = 0
for end_data in state_gaps:
    plt.plot(state_time[begin_data:end_data], height[begin_data:end_data], alpha=0.8, color=red)
    begin_data = end_data+1

# label hack
plt.plot([state_time[0],state_time[0]],[0,0], color=red, alpha=0.8, label="Instantaneous Geocentric Altitude")

plt.plot([mar1, mar31], [avg_height, avg_height], 'k--', alpha=0.4, label="Mean Geocentric Altitude (%0.1f) km" % avg_height)

ax1.xaxis.set_major_formatter(date_fmt_day)
plt.ylim([400,425])
plt.xlim(march)
ax1.legend(loc=1)
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(24,8))
plt.title(r"ISS Velocity Over Time")
plt.ylabel(r"ECI Velocity [m/s]")
plt.xlabel(r"State Vector Date [Month Day]")


begin_data = 0
for end_data in state_gaps:
    ax1.plot(state_time[begin_data:end_data], velocity[begin_data:end_data], color=red, alpha=0.75)
    begin_data = end_data+1

# label hack
ax1.plot([state_time[0],state_time[0]], [0,0], color=red, alpha=0.75, label="Instantaneous ECI Velocity")

ax1.plot([mar1, mar31], [avg_velocity, avg_velocity], 'k--', alpha=0.4, label="Mean ECI Velocity (%0.1f) m/s" % avg_velocity)
ax1.plot([state_time[0],state_time[0]], [0,0], color=blue, linestyle='--', alpha=0.3, label="Geocentric Altitude")


ax1.xaxis.set_major_formatter(date_fmt_day)
plt.ylim([7655,7680])
ax1.legend(loc=1)

ax2 = ax1.twinx()
begin_data = 0
for end_data in state_gaps:
    ax2.plot(state_time[begin_data:end_data], height[begin_data:end_data], color=blue, linestyle='-.', alpha=0.3)
    begin_data = end_data+1
ax2.grid(b=False)
plt.ylim([400,425])
plt.ylabel(r"Geocentric Altitude [km]")

plt.xlim(march)
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(24,8))
plt.title(r"ISS Instantaneous Orbital Elements: Semi-Major Axis")
plt.ylabel(r"Semi-Major Axis [km]")
plt.xlabel(r"State Vector Date [Month Day]")

begin_data = 0
for end_data in state_gaps:
    ax1.plot(state_time[begin_data:end_data], a[begin_data:end_data], color=red, alpha=0.75)
    begin_data = end_data+1

# label hack
ax1.plot([state_time[0],state_time[0]], [0,0], color=red, alpha=0.75, label="Instantaneous Semi-Major Axis")

ax1.plot([mar1, mar31], [avg_a, avg_a], 'k--', alpha=0.4, label="Mean Semi-Major Axis (%0.1f) km" % avg_a)

ax1.xaxis.set_major_formatter(date_fmt_day)
ax1.legend(loc=1)
plt.ylim([RADIUS_EARTH+404, RADIUS_EARTH+421])
plt.xlim(march)
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(24,8))
plt.title(r"ISS Instantaneous Orbital Elements: Inclination")
plt.ylabel(r"Obrital Inclination [${}^0$]")
plt.xlabel(r"State Vector Date [Month Day]")

begin_data = 0
for end_data in state_gaps:
    ax1.plot(state_time[begin_data:end_data], incl[begin_data:end_data], color=red, alpha=0.75)
    begin_data = end_data+1

# label hack
ax1.plot([state_time[0],state_time[0]], [0,0], color=red, alpha=0.75, label="Inclination")

ax1.xaxis.set_major_formatter(date_fmt_day)
plt.ylim([51.6-0.1,51.6+0.2])
plt.xlim(march)
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(24,8))
plt.title(r"ISS Instantaneous Orbital Elements: Eccentricity")
plt.ylabel(r"Obrital Inclination")
plt.xlabel(r"State Vector Date [Month Day]")

begin_data = 0
for end_data in state_gaps:
    ax1.plot(state_time[begin_data:end_data], e[begin_data:end_data], color=red, alpha=0.75)
    begin_data = end_data+1

# label hack
ax1.plot([state_time[0],state_time[0]], [0,0], color=red, alpha=0.75, label="Instantaneous Eccentricity")

ax1.plot([mar1, mar31], [avg_e, avg_e], 'k--', alpha=0.4, label="Mean Eccentricity (%0.5f)" % avg_e)

ax1.xaxis.set_major_formatter(date_fmt_day)
ax1.legend(loc=1)
plt.ylim([0,0.002])
plt.xlim(march)
plt.show()

## Sort By Orbit

In [ ]:
orbits = []
for i, x_1 in enumerate(state_x[1:2000]):
    x_0 = state_x[i]
    
    if x_1 > 0 and x_0 <=0:
        orbits.append(i+1)

In [ ]:
def func(x, a, b, c):
    #p, sm_a, ecc, inc, omega, argp, nu, m, arglat, truelon, lonper
    return a + b*x + c*x*x

In [ ]:
fig, ax1 = plt.subplots(figsize=(24,8))
plt.title(r"ISS State Vector J2000 X,Y,Z Position Over Time")
plt.ylabel(r"J2000 Coordinate [km]")
plt.xlabel(r"State Vector Date [Month Day]")

begin_data = 0
for end_data in state_gaps:
    if begin_data > orbits[-1]: break
    plt.plot(state_time[begin_data:end_data], state_x[begin_data:end_data], color=red, alpha=0.75)
    plt.plot(state_time[begin_data:end_data], state_y[begin_data:end_data], color=blue, alpha=0.75)
    plt.plot(state_time[begin_data:end_data], state_z[begin_data:end_data], color=green, alpha=0.75)
    begin_data = end_data+1

for orbit in orbits:
    plt.plot([state_time[orbit], state_time[orbit]], [-10000,10000], 'k--', alpha=0.5, lw=2)

# hack for labels
plt.plot([state_time[0],state_time[0]], [-100000,-110000], color=red, alpha=0.8, label="X")
plt.plot([state_time[0],state_time[0]], [-100000,-110000], color=blue, alpha=0.8, label="Y")
plt.plot([state_time[0],state_time[0]], [-100000,-110000], color=green, alpha=0.8, label="Z")

ax1.xaxis.set_major_formatter(date_fmt_long)
plt.setp(ax1.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.ylim([-10e3,10e3])
ax1.legend(loc=1, title="Coordinate:")
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(24,8))
plt.title(r"ISS Semi-Major Axis By Orbit")
plt.ylabel(r"Semi-Major Axis [km]")
plt.xlabel(r"State Vector Date [Month Day]")

begin_data = 0
for end_data in state_gaps:
    if begin_data > orbits[-1]: break
    ax1.plot(state_time[begin_data:end_data], a[begin_data:end_data], color=red, alpha=0.75)
    begin_data = end_data+1

for orbit in orbits:
    plt.plot([state_time[orbit], state_time[orbit]], [-10000,10000], 'k--', alpha=0.5, lw=2)

# hack for labels
plt.plot([state_time[0],state_time[0]], [-100000,-110000], color=red, alpha=0.75, label="Instantanious Semi-Major Axis")

ax1.xaxis.set_major_formatter(date_fmt_long)
plt.setp(ax1.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.ylim([RADIUS_EARTH+404, RADIUS_EARTH+421])
ax1.legend(loc=1)
plt.show()

--------------------------------------------------------------------------------------

This article was written in
[Jupyter Notebooks](http://jupyter.org/).
[View the original document](http://nbviewer.jupyter.org/github/natronics/ISS-TLE-Propagation/blob/gh-pages/index.ipynb) or download the raw file: [index.ipynb](https://raw.githubusercontent.com/natronics/ISS-TLE-Propagation/gh-pages/index.ipynb)

Did I make a mistake? Have something to add? You can
[create an issue](https://github.com/natronics/ISS-TLE-Propagation/issues)
, or
[build this page yourself](https://github.com/natronics/ISS-TLE-Propagation/blob/gh-pages/README.markdown)
and
[send a pull request](https://help.github.com/categories/collaborating-on-projects-using-pull-requests/).

<a rel="license" href="https://creativecommons.org/licenses/by-nc/4.0/"><img class="wrap" alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/80x15.png" /></a> &nbsp; This <span xmlns:dct="https://purl.org/dc/terms/" href="https://purl.org/dc/dcmitype/InteractiveResource" rel="dct:type">work</span> by <a xmlns:cc="https://creativecommons.org/ns#" href="https://github.com/natronics" property="cc:attributionName" rel="cc:attributionURL">natronics</a> is licensed under a <a rel="license" href="https://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

#### Footnotes